https://py.iceberg.apache.org/#installation

In [27]:
#pip install --upgrade pip

In [2]:
# pip install "pyiceberg[s3fs,hive]"

In [4]:
# pip install s3fs

In [8]:
# pip install adlfs

In [9]:
#pip install gcsfs

In [10]:
#pip install pyarrow

For the sake of demonstration, we'll configure the catalog to use the SqlCatalog implementation, which will store information in a local sqlite database. We'll also configure the catalog to store data files in the local filesystem instead of an object store. This should not be used in production due to the limited scalability.

In [61]:
#Create a temporary location for Iceberg:
#using jupyter nb:
!mkdir -p ./tmp/warehouse
#using vscode:
#mkdir ./tmp/warehouse

In [62]:
#conda install sqlalchemy

In [1]:
#Open a Python 3 REPL to set up the catalog:
from pyiceberg.catalog.sql import SqlCatalog

warehouse_path = "./tmp/warehouse"
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

Write a PyArrow dataframe
Let's take the Taxi dataset, and write this to an Iceberg table.

First download one month of data:

In [64]:
#jupyter exec:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o ./tmp/yellow_tripdata_2023-01.parquet

#vscode
#curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o /tmp/yellow_tripdata_2023-01.parquet


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  2932k      0  0:00:15  0:00:15 --:--:-- 2978k 3792k    0     0  2824k      0  0:00:16  0:00:01  0:00:15 2824k


In [65]:
# Load it into your PyArrow dataframe:
import pyarrow.parquet as pq

df = pq.read_table("./tmp/yellow_tripdata_2023-01.parquet")

In [66]:
#Create a new Iceberg table:
catalog.create_namespace("default")

table = catalog.create_table(
    "default.taxi_dataset",
    schema=df.schema,
)

In [67]:
# Append the dataframe to the table:
table.append(df)
len(table.scan().to_arrow())
#3066766 rows will been written to the table.

3066766

In [68]:
# Now generate a tip-per-mile feature to train the model on:
import pyarrow.compute as pc

df = df.append_column("tip_per_mile", pc.divide(df["tip_amount"], df["trip_distance"]))

In [69]:
#Evolve the schema of the table with the new column:
with table.update_schema() as update_schema:
    update_schema.union_by_name(df.schema)

In [70]:
#And now we can write the new dataframe to the Iceberg table:
table.overwrite(df)
print(table.scan().to_arrow())

pyarrow.Table
VendorID: int64
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
tip_per_mile: double
----
VendorID: [[2,2,2,1,2,...,2,2,2,2,2]]
tpep_pickup_datetime: [[2023-01-01 00:32:10.000000,2023-01-01 00:55:08.000000,2023-01-01 00:25:04.000000,2023-01-01 00:03:48.000000,2023-01-01 00:10:29.000000,...,2023-01-31 23:58:34.000000,2023-01-31 23:31:09.000000,2023-01-31 23:01:05.000000,2023-01-31 23:40:00.000000,2023-01-31 23:07:32.000000]]
tpep_dropoff_datetime: [[2023-01-01 00:40:36.000000,2023-01-01 01:01:27.000000,2023-01-01 00:37:49.000000,2023-01-01 00:13:25.000000,2023-01-01 00:21:19.000000,...,2023-02-01 00:12:33.000000,

In [71]:
#And we can see that 2371784 rows have a tip-per-mile:
df = table.scan(row_filter="tip_per_mile > 0").to_arrow()
len(df)


2371784

In [72]:
type(df)

pyarrow.lib.Table

In [73]:
#Explore Iceberg data and metadata files
#Since the catalog was configured to use the local filesystem, we can explore how Iceberg saved data and metadata files from the above operations.
# using jupyter nb
!find ./tmp/warehouse/

# using vscode
#find /tmp/warehouse/

./tmp/warehouse/
./tmp/warehouse//pyiceberg_catalog.db
./tmp/warehouse//default.db
./tmp/warehouse//default.db/taxi_dataset
./tmp/warehouse//default.db/taxi_dataset/data
./tmp/warehouse//default.db/taxi_dataset/data/00000-0-979bdd8b-2188-4d83-9c8f-d4d98c09031d.parquet
./tmp/warehouse//default.db/taxi_dataset/data/00000-0-82dae1e5-8b76-4cf6-95ef-9876d06db6a5.parquet
./tmp/warehouse//default.db/taxi_dataset/metadata
./tmp/warehouse//default.db/taxi_dataset/metadata/25a6b7bd-3b22-444c-a9c6-a939a1ad9f00-m0.avro
./tmp/warehouse//default.db/taxi_dataset/metadata/00003-5d23fd97-e3b4-43e2-bceb-3778a5ec3c38.metadata.json
./tmp/warehouse//default.db/taxi_dataset/metadata/25a6b7bd-3b22-444c-a9c6-a939a1ad9f00-m1.avro
./tmp/warehouse//default.db/taxi_dataset/metadata/00000-df6e79f2-5f56-428b-86de-b71d94df97ad.metadata.json
./tmp/warehouse//default.db/taxi_dataset/metadata/00002-4d4b5473-ba35-4f2b-b68f-b157c80a37c8.metadata.json
./tmp/warehouse//default.db/taxi_dataset/metadata/snap-6434905510831924

# pip install pyspark

In [1]:
#!pip install pyiceberg


In [13]:
spark.stop()

In [27]:
from pyspark.sql import SparkSession

# Create Spark session with Iceberg configuration
spark = SparkSession.builder \
    .appName("IcebergCatalog") \
    .config("spark.sql.catalog.local_warehouse", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local_warehouse.type", "hadoop") \
    .config("spark.sql.catalog.local_warehouse.warehouse", "file:///Users/victorgalan/vscode_projects/iceberg-spark/tmp/warehouse") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.0.0") \
    .getOrCreate()

catalog_name = "pyiceberg_catalog"

# Set the catalog configuration
spark.conf.set(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set(f"spark.sql.catalog.{catalog_name}.type", "hadoop")
spark.conf.set(f"spark.sql.catalog.{catalog_name}.warehouse", "file:///Users/victorgalan/vscode_projects/iceberg-spark/tmp/warehouse")

# Show Databases in the Catalog
databases = spark.sql(f"SHOW DATABASES IN {catalog_name}")
databases.show()

# Create a database
spark.sql(f"CREATE DATABASE {catalog_name}.example_db")

# Show the newly created database
databases = spark.sql(f"SHOW DATABASES IN {catalog_name}")
databases.show()

# Create a table in the new 


Py4JJavaError: An error occurred while calling o121.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'pyiceberg_catalog': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:2079)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala

In [4]:
spark

In [5]:
# Show Databases in the Catalog
databases = spark.sql(f"SHOW DATABASES IN {catalog_name}")
databases.show()

Py4JJavaError: An error occurred while calling o35.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'warehouse': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:2079)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:30)
	at 

In [ ]:
# Assuming there is a database named 'default' in the catalog, show tables in this database
tables = spark.sql(f"SHOW TABLES IN {catalog_name}.default")
tables.show()

In [119]:
spark.sql(f"SHOW TABLES IN {catalog_name}").show()


Py4JJavaError: An error occurred while calling o698.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'default': org.apache.iceberg.spark.SparkSessionCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:2079)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:30

In [30]:
from pyspark.sql import SparkSession

# Create Spark session with Iceberg configuration
spark = SparkSession.builder \
    .appName("IcebergCatalog") \
    .config("spark.sql.catalog.local_warehouse", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local_warehouse.type", "jdbc") \
    .config("spark.sql.catalog.local_warehouse.uri", "jdbc:sqlite:/Users/victorgalan/vscode_projects/iceberg-spark/tmp/warehouse/iceberg.db") \
    .config("spark.sql.catalog.local_warehouse.jdbc.driver", "org.sqlite.JDBC") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.0.0,org.xerial:sqlite-jdbc:3.36.0.3") \
    .getOrCreate()

catalog_name = "pyiceberg_catalog.db"

# Show Databases in the Catalog
databases = spark.sql(f"SHOW DATABASES IN {catalog_name}")
databases.show()

# Create a database
spark.sql(f"CREATE DATABASE {catalog_name}.example_db")

# Show the newly created database
databases = spark.sql(f"SHOW DATABASES IN {catalog_name}")
databases.show()

# Create a table in the new database
spark.sql(f"""
CREATE TABLE {catalog_name}.example_db.example_table (
    id INT,
    name STRING,
    data STRING
) USING iceberg
""")

# Show tables in the database
tables = spark.sql(f"SHOW TABLES IN {catalog_name}.example_db")
tables.show()

# Insert some data into the table
spark.sql(f"""
INSERT INTO {catalog_name}.example_db.example_table VALUES
(1, 'Alice', 'Data 1'),
(2, 'Bob', 'Data 2'),
(3, 'Charlie', 'Data 3')
""")

# Query the table
data = spark.sql(f"SELECT * FROM {catalog_name}.example_db.example_table")
data.show()


Py4JJavaError: An error occurred while calling o121.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'pyiceberg_catalog': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:2079)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala

In [32]:
from pyspark.sql import SparkSession

# Create Spark session with Iceberg and SQLite configuration
spark = SparkSession.builder \
    .appName("IcebergCatalogSQLite") \
    .config("spark.sql.catalog.local_warehouse", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local_warehouse.type", "hive") \
    .config("spark.sql.catalog.local_warehouse.uri", "thrift://localhost:9083") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark3-runtime:0.15.0") \
    .getOrCreate()

catalog_name = "local_warehouse"

# Show databases in the catalog
print("Databases in the catalog:")
databases = spark.sql(f"SHOW DATABASES IN {catalog_name}")
databases.show()

# Create a new database
database_name = "example_db"
print(f"Creating database: {database_name}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{database_name}")

# Show the newly created database
print("Databases after creating the new database:")
databases = spark.sql(f"SHOW DATABASES IN {catalog_name}")
databases.show()

# Create a new table in the new database
table_name = "example_table"
print(f"Creating table: {table_name} in database: {database_name}")
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.{database_name}.{table_name} (
    id INT,
    name STRING,
    data STRING
) USING iceberg
""")

# Show tables in the new database
print(f"Tables in database: {database_name}")
tables = spark.sql(f"SHOW TABLES IN {catalog_name}.{database_name}")
tables.show()

# Insert some data into the table
print(f"Inserting data into table: {table_name}")
spark.sql(f"""
INSERT INTO {catalog_name}.{database_name}.{table_name} VALUES
(1, 'Alice', 'Data 1'),
(2, 'Bob', 'Data 2'),
(3, 'Charlie', 'Data 3')
""")

# Query the table
print(f"Querying data from table: {table_name}")
data = spark.sql(f"SELECT * FROM {catalog_name}.{database_name}.{table_name}")
data.show()

# Stop Spark session
spark.stop()


24/06/29 20:59:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Databases in the catalog:


Py4JJavaError: An error occurred while calling o121.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'local_warehouse': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:2079)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:3